# Kinematics for Driving

<a href="https://colab.research.google.com/github/gtbook/robotics/blob/main/S62_driving_actions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```{index} action; Ackermann steering
```

> Cars are more complex than differential-drive robots: they cannot turn in place, and they typically have front-wheel steering.

<img src="Figures6/S62-Autonomous_Vehicle_with_LIDAR_and_cameras-06.jpg" alt="Splash image with steampunk autonomous car" width="40%" align=center style="vertical-align:middle;margin:10px 0px">

In this section we introduce a kinematic model for cars.
Unlike the differential-drive robots of the previous chapter, cars have four wheels, two of which are used for steering,
and two of which are used to induce linear motion (either the two rear or two front wheels, except in the case of four-wheel drive vehicles).
Therefore, we might choose to model the car as having two inputs: a steering angle and an acceleration.
While this might be a more accurate physical model, we will use a slightly simplified model, in which the
forward speed and turning rate are the inputs.
Unless the car is traveling at high speed, or experiencing large accelerations, this model is
accurate enough for most purposes.

## Car kinematics

<figure id="fig:TransformComposition2">
<img src="https://github.com/gtbook/robotics/blob/main/Figures6/car1.png?raw=1" style="width:8cm" alt="">
<figcaption>Coordinate frame for a car-like vehicle. </figcaption>
</figure>

We will assume that our car has front-wheel steering, and that forward velocity is achieved by actuation
of the rear wheels.
We assign a body-attached frame to the car with origin at the midpoint of the rear axle and $x$-axis pointing forward,
as shown in Figure [1](#fig:TransformComposition2).
If the tires roll without slipping, the instantaneous velocity of the car is always in the direction of
the body-attached $x$-axis.
We denote the forward speed by $v_\mathrm{car}$ (note that $v_\mathrm{car}$ denotes the *scalar* speed of the car,
and not the car velocity, which is a *vector* quantity).

```{index} turning circle
```
At first glance, it may appear that the two front wheels have the same orientation with respect to the car frame.
This is not the case, however.
As the origin of the car frame traces out a curve in the plane, in order for all wheels to roll without
slipping, their instantaneous velocities must be tangent to a set of circles that share a common origin,
called the center of the **turning circle**.
For the rear wheels, because they share a common axle and maintain a fixed orientation with respect to the car frame,
the axle coincides with the radius of concentric circles,
and each of the rear wheels has instantaneous velocity that is perpendicular to this radius.
For the front wheels, the inner wheel follows a circle with smaller radius than the outside wheel.

<figure id="fig:AckermannSteering1">
<img src="https://github.com/gtbook/robotics/blob/main/Figures6/ackermann1.PNG?raw=1"  style="width:8cm" alt="">
<figcaption>Ackermann steering.</figcaption>
</figure>

```{index} Ackermann steering
```
This kind of steering for the front wheels is called **Ackermann steering**, as illustrated in Figure [2](#fig:AckermannSteering1).
The physical mechanism required to implement Ackermann steering is slightly complex,
but happily we can model the system by using a single *virtual wheel* placed
at the midpoint between the two front wheels, rolling in a direction perpendicular to the line from
the center of the turning circle to this midpoint.
We denote by $\phi$ the angle from the car $x$-axis to the forward direction of this virtual wheel,
as shown in Figure [3](#fig:AckermannSteering2).

<figure id="fig:AckermannSteering2">
<img src="https://github.com/gtbook/robotics/blob/main/Figures6/car2.png?raw=1" style="width:8cm" alt="">
<figcaption>Virtual wheel used to model Ackermann steering.</figcaption>
</figure>

The configuration of the car, shown in Figure [4](#fig:CarConfigurationParams),
can be represented as $q = (x,y,\theta, \phi)$, in which $x,y$ denotes the position
of the origin of the car frame, $\theta$ denotes the orientation of car frame (the angle from the world $x$-axis
to the car frame $x$-axis), and $\phi$ denotes the steering angle.
The inputs to the car are
* $u_1= v_\mathrm{car}$, the linear speed of the car, and
* $u_2 = \dot{\phi}$, the turning rate of the virtual front wheel.

<figure id="fig:CarConfigurationParams">
<img src="https://github.com/gtbook/robotics/blob/main/Figures6/car3.png?raw=1" style="width:12cm" alt="">
<figcaption>Configuration parameters for the car.</figcaption>
</figure>

## Differential Kinematics

The differential kinematics for the car are specified as an equation,
$\dot{q} = f(q, u_1, u_2)$.
The first two components, $\dot{x}$ and $\dot{y}$ are straightforward functions
of  $v_\mathrm{car}$ and $\theta$
\begin{equation}
\begin{aligned}
\dot{x} &=  v_\mathrm{car} \cos \theta = u_1 \cos \theta \\
\dot{y} &=  v_\mathrm{car} \sin \theta = u_1 \sin \theta 
\end{aligned}
\end{equation}
The steering angle is also straightforward:
\begin{equation}
\dot{\phi} = u_2
\end{equation}
Note that using this model, we cannot directly control the car's orientation $\theta$.
We cannot even directly choose the steering angle $\phi$.
We have direct control only over $\dot{\phi}$.

The equation for $\dot{\theta}$ is slightly more complex.
From the Figure [5](#fig:CarRelevantVelocities), we observe the following
\begin{equation}
\begin{aligned}
v_\mathrm{wheel} \cos \phi &=  v_\mathrm{car}  \\
v_\mathrm{wheel} \sin \phi &=  v_\mathrm{tangent}  \\
\end{aligned}
\end{equation}
Combining these to eliminate $v_\mathrm{wheel}$ we obtain
\begin{equation}
\frac{v_\mathrm{car}}{\cos \phi} \sin \phi = v_\mathrm{car} \tan \phi = v_\mathrm{tangent}
\end{equation}
Now, using $L \dot{\theta} = v_\mathrm{tangent}$ (where $L$ is the baseline distance between front and rear axles),
we obtain
\begin{equation}
\dot{\theta} = v_\mathrm{car} \frac{1}{L} \tan \phi = u_1 \frac{1}{L} \tan \phi
\end{equation}

<figure id="fig:CarRelevantVelocities">
<img src="https://github.com/gtbook/robotics/blob/main/Figures6/car4.png?raw=1" style="width:12cm" alt="">
<figcaption>Relevant velocities for computing the differential kinematics of the car.</figcaption>
</figure>

Collecting these terms together, we can write the differential kinematics
in matrix form as:

\begin{equation}
\begin{bmatrix}
\dot{x}\\
\dot{y}\\
\dot{\theta}\\
\dot{\phi}
\end{bmatrix}
=
\begin{bmatrix}
\cos \theta\\
\sin \theta \\
\frac{1}{L} \tan \phi\\
0
\end{bmatrix}
u_1
+
\begin{bmatrix}
0\\
0\\
0\\
1
\end{bmatrix}
u_2
\end{equation}